In [6]:
print("Hello world")

Hello world


In [20]:
import tweepy
from deep_translator import GoogleTranslator

token = ""
client = tweepy.Client(bearer_token=token)

# User Id from twitter profile
userid = "44196397"
paginator = tweepy.Paginator(client.get_users_tweets, id=userid, max_results=8)

print("Elon Musk Last 8 Tweets")
count = 1
while count <= 8 :
    for tweet in paginator.flatten(limit=8):
        translated = GoogleTranslator(source='auto', target='nl').translate(str(tweet))
        
        print("Tweet", count)
        print(translated, "\n")

        count = count + 1



Elon Musk Last 8 Tweets
Tweet 1
Gezouten boter is geweldig, maar moet bij kamertemperatuur worden bewaard https://t.co/8RSr340JB8 

Tweet 2
@corey_aronson @CathieDWood Daling 0,25% 

Tweet 3
@CathieDWood Precies, dit is niet subtiel of geheim 

Tweet 4
@WholeMarsBlog Lagen op lagen van neurale netwerken, steeds hoger 

Tweet 5
https://t.co/xMoL9wdXh6 

Tweet 6
@HuobiGlobal Subtiel 

Tweet 7
@HuobiGlobal 

Tweet 8
Let op, we werken eraan om het fluitje veel luider te laten klinken 

